In [65]:
# 남은 할일 
# 기사 제목 끌고 와서 라벨링 해준다
# 그 데이터 가지고 정확도를 예측한다. 이 때 사용될 건 accuracy, f1 score 

import pandas as pd
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

search = input("회사명(csv 파일명) : ")
num = int(input("몇 번째 테스트? "))
# csv
df = pd.read_csv(f'csv/{search} 뉴스 제목{num}.csv')

# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# df['content'] = df['title']

# title 열에 대해 형태소 분석 적용
df['title'] = df['title'].apply(tokenize)

# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment
df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'

# train할 label은 제목을 읽고 내가 직접 라벨링
# test할 label은 0.5를 기준으로 sentiment가 0.5보다 크면 1, 작으면 0으로 기준 세움
df['label'] = df['sentiment'].apply(convert_sentiment)
df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

회사명(csv 파일명) : 삼성전자
몇 번째 테스트? 1


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [66]:
# csv 파일로 저장
df.to_csv(f'csv/{search}_test_data_{num}.csv', index=False)

# 결과 확인
print(df['title'])
print(df['sentiment'])

0     [삼성전자, ,, 갤럭시·노트북·TV서, ', 자, 가, 수, 이, 리, ', 국내...
1                    [삼성전자, 갤럭시, ,, 이제, 내손, 으로, 고치, ㄴ다]
2        [[, 특징주, ], 삼성전자·SK하이닉스,, 52주, 신, 고, 가, 아, 기록]
3            [", 청년, 에, 일경험"…SK하이닉스·삼성전자, 등, 2100명, 모집]
4           [작년, 주주, 배당액, 1위, 는, 삼성전자…총, 배당금, 은, 6.7%↓]
5       [“고장난, 갤럭시, 직접, 고친다”…삼성전자,, ‘자, 가, 수리’, 국내, 도입]
6     [", 이번, 엔, 중동"…삼성전자,, 두바, 이, 리조트, 서, 2023년형, T...
7            [삼성전자, ,, 두바이, 서, 2023년형, 네오QLED, 신제품, 선봬]
8     [[, 특징주, ], 날개, 단, 삼성전자‧SK하이닉스…또, 52주, 최고, 가, 경신]
9     [", 이제, 집, 에서, 고치, 세, 요, ", ..., 삼성전자, ,, ', 자...
10      ["머스크, 불, 만, 터트리, ㄹ, 지경인데"…삼성전자·하이닉스, 웃, 는, 이유]
11               [삼성전자, 신, 고, 가, 아, 행진…"8만전자, 갈, ㄴ다, "]
12       [", 삼성전자, ,, 책임경영, 필요성↑…이재용, 등기이사, 복귀, 가능성, "]
13          [삼성전자, ,, 외국, 이, ㄴ, 폭풍매수, 에, '7만전자, 굳히기, ']
14       [삼성전자, ,, 7.1만원, 진입…반도체株, 훈풍, 지, 속, [, 특징주, ]]
15    [", 삼성전자, 가, 또, ?"..., 신제품, 에, 적용, 하, ㄴ, ', 첨단...
16        [로이터, “삼성전자·LG전자,, 베트남, 정부, 에, 최저한세, 보상, 요구”]
17            [삼성전자, 신, 고, 가, 아, 행진…코스피,, 열, ㄴ, 

# 아래는 train_set 업데이트 할 때만 실행

In [67]:
# csv
df = pd.read_csv(f'csv/samsung_data_set.csv',encoding='cp949')

# content 열에 대해 형태소 분석 적용
df['title'] = df['title'].apply(tokenize)

df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

In [68]:
df.to_csv(f'csv/samsung_train.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'csv/samsung_train.csv'

In [ ]:
df